In [29]:
from enum import Enum
import heapq
from dataclasses import dataclass
from typing import Self, Iterable
from functools import total_ordering

In [30]:
class Direction(Enum):
    N = 0
    E = 1
    S = 2
    W = 3

    def opposite(self) -> Self:
        return Direction((self.value+2)%4)
    
    def cont(self) -> Iterable[Self]:
        return (d for d in Direction if d != self.opposite())

@dataclass(frozen=True)
class State:
    x: int
    y: int
    d: Direction
    length: int

@total_ordering
@dataclass
class Prio:
    state: State
    p: int

    def __lt__(self, other: Self) -> bool:
        return self.p < other.p

In [47]:
def neighbors(state: State, lenx: int, leny: int) -> list[State]:
    new: list[State] = []
    for d in state.d.cont():
        if d == state.d:
            if state.length == 2:
                continue
            else:
                l = state.length + 1
        else:
            l = 0

        match d:
            case Direction.N:
                if state.y == 0:
                    continue
                new.append(State(state.x, state.y-1, d, l))
            case Direction.E:
                if state.x == lenx-1:
                    continue
                new.append(State(state.x+1, state.y, d, l))
            case Direction.S:
                if state.y == leny-1:
                    continue
                new.append(State(state.x, state.y+1, d, l))
            case Direction.W:
                if state.x == 0:
                    continue
                new.append(State(state.x-1, state.y, d, l))
    return new

In [58]:
def part1(text: list[str]) -> int:
    grid = [[int(t) for t in l.strip()] for l in text]
    lenx = len(grid[0])
    leny = len(grid)
    frontier = [Prio(State(0, 0, Direction.E, 0), 0), Prio(State(0, 0, Direction.S, 0), 0)]
    heapq.heapify(frontier)
    cost = {State(0, 0, Direction.E, 0): 0, State(0, 0, Direction.S, 0): 0}
    while len(frontier) > 0:
        current = heapq.heappop(frontier).state
        if current.x == lenx-1 and current.y == leny-1:
            break
        for n in neighbors(current, lenx, leny):
            ncost = cost[current] + grid[n.y][n.x]
            if n not in cost.keys() or ncost < cost[n]:
                cost[n] = ncost
                heapq.heappush(frontier, Prio(n, ncost))
    r = float('inf')
    for k, v in cost.items():
        if k.x == lenx-1 and k.y == leny-1:
            r = min(r, v)
    return r

In [27]:
with open('test.txt', 'rt') as f:
    test = f.readlines()

In [59]:
part1(test)

102

In [54]:
with open('input', 'rt') as f:
    inp = f.readlines()

In [60]:
with open('output1', 'wt') as f:
    f.write(str(part1(inp)))